<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%A6%9D%EB%B6%84/(%EB%85%B8%EC%9D%B4)_%EC%9B%90%EB%B3%B8I%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본			   G_inc_so_model_노이2.pt
 산업분류자동화		   G_inc_so_model_노이.pt
'Colab Notebooks'	  'My Drive'
 G_inc_again_so_model.pt   Screenshot_20220401-001411_Chrome.jpg


In [3]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/spell_check/I_spell_check.csv', sep = ",", encoding = "UTF-8")

## 데이터 전처리

In [4]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000007,I,56,561,음식점에서 접객시설을 갖추고 참치 회(일본식)
1,id_0000012,I,56,561,음식점 접객시설을 갖추고 냉면
2,id_0000014,I,56,561,음식점에서 접객시설을 갖추고 회
3,id_0000023,I,56,562,유흥주점에서 무도 시설을 갖추고 술판매
4,id_0000025,I,56,561,"음료점에서 빙수, 아이스크림, 커피, 디저트 제공 커피. 빙수 등을 제공"
...,...,...,...,...,...
187420,id_0999978,I,56,561,사업장 구내식당에서 회사와 계약을 맺고 음식 제공
187421,id_0999981,I,56,562,"주점에서 접객시설 갖추고 주류, 안주류"
187422,id_0999987,I,56,561,음식점에서 육류 제공
187423,id_0999988,I,55,551,펜션 운영업


In [5]:
df['digit_3'].value_counts().sort_index()

551     12549
559      1417
561    123703
562     49756
Name: digit_3, dtype: int64

## **# 데이터 증분**

In [6]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [7]:
df0=df.loc[(df['digit_3'] == 551)]  #*10
df1=df.loc[(df['digit_3'] == 559)] #*100
df2=df.loc[(df['digit_3'] == 561)]  
df3=df.loc[(df['digit_3'] == 562)]  #*2   

In [8]:
df0=df0.reset_index(drop=True)
df1=df1.reset_index(drop=True)
df2=df2.reset_index(drop=True)
df3=df3.reset_index(drop=True)

In [9]:
df1.head()

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000073,I,55,559,사업장에서 일반인을 대상으로 숙박시설(고시원)
1,id_0002089,I,55,559,고시원에서 이용 고객에게 숙박시설 제공
2,id_0002556,I,55,559,고시원에서 일반 고객 대상으로 숙박 서비스
3,id_0003146,I,55,559,고시원에서 일반 고객 대상으로 숙식제공 서비스
4,id_0004122,I,55,559,고시원에서 일반 고객들에게 임대 숙박 서비스


In [10]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df3['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=1)
  id_list=['p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n])]
  digit_1_list=[df3['digit_1'][n], df3['digit_1'][n]]
  digit_2_list=[df3['digit_2'][n], df3['digit_2'][n]]
  digit_3_list=[df3['digit_3'][n], df3['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df3_to_insert = pd.DataFrame(data_to_insert)
  df3= df3.append(df3_to_insert, ignore_index=True)
  n=n+1

In [11]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df0['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=9)
  id_list=[str(df0['AI_id'][n]), 'p'+str(df0['AI_id'][n]), 'p'+str(df0['AI_id'][n]), 'p'+str(df0['AI_id'][n]), 'p'+str(df0['AI_id'][n]), 'p'+str(df0['AI_id'][n]), 'p'+str(df0['AI_id'][n]), 'p'+str(df0['AI_id'][n]), 'p'+str(df0['AI_id'][n]), 'p'+str(df0['AI_id'][n])]
  digit_1_list=[df0['digit_1'][n], df0['digit_1'][n], df0['digit_1'][n], df0['digit_1'][n], df0['digit_1'][n], df0['digit_1'][n], df0['digit_1'][n], df0['digit_1'][n], df0['digit_1'][n], df0['digit_1'][n]]
  digit_2_list=[df0['digit_2'][n], df0['digit_2'][n], df0['digit_2'][n], df0['digit_2'][n], df0['digit_2'][n], df0['digit_2'][n], df0['digit_2'][n], df0['digit_2'][n], df0['digit_2'][n], df0['digit_2'][n]]
  digit_3_list=[df0['digit_3'][n], df0['digit_3'][n], df0['digit_3'][n], df0['digit_3'][n], df0['digit_3'][n], df0['digit_3'][n], df0['digit_3'][n], df0['digit_3'][n], df0['digit_3'][n], df0['digit_3'][n]]

  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df0_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df0_to_insert, ignore_index=True)
  n=n+1

df0=temp

In [12]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df1['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=99)
  id_list=[str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n])]
  digit_1_list=[df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n]]
  digit_2_list=[df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n]]
  digit_3_list=[df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n]]

  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df1_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df1_to_insert, ignore_index=True)
  n=n+1

df1=temp

기존 데이터 합치기

In [13]:
df=pd.concat([df0, df1, df2, df3])

In [14]:
df['digit_3'].value_counts().sort_index()

551.0    125490
559.0    141700
561.0    123703
562.0    149268
Name: digit_3, dtype: int64

In [15]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000041,I,55.0,551.0,여관업
1,pid_0000041,I,55.0,551.0,여관업
2,pid_0000041,I,55.0,551.0,여관업
3,pid_0000041,I,55.0,551.0,여관업
4,pid_0000041,I,55.0,551.0,여관업
...,...,...,...,...,...
149263,pid_0999910,I,56.0,562.0,"주점에서 접객시설을 갖추고 술, 안주에 공"
149264,pid_0999970,I,56.0,562.0,접객시설을 커피 등
149265,pid_0999970,I,56.0,562.0,커피숍에서 접객시설을 갖추고 커피 등 판매
149266,pid_0999981,I,56.0,562.0,"주점에서 갖추고 안주류 주류, 접객시설"


In [16]:
df.to_csv('df_I_inc.csv', index=False, encoding='utf-8-sig')